#### Paso 0

Lectura de los datasets:

In [1]:
# # Importar las librerias necesarias
# import zipfile
# import pandas as pd
# import os

# # Rutas de los archivos ZIP
# train_zip_path = os.path.join("../files/input", "train_data.csv.zip")
# test_zip_path = os.path.join("../files/input", "test_data.csv.zip")

# # Función para leer un CSV dentro de un archivo ZIP
# def read_csv_from_zip(zip_path, file_name = None):
#     with zipfile.ZipFile(zip_path, 'r') as z:
#         # Si no se especifica el nombre del archivo, toma el primero en el ZIP
#         file_name = file_name or z.namelist()[0]
#         with z.open(file_name) as f:
#             return pd.read_csv(f)

# # Leer los datasets
# train_data = read_csv_from_zip(train_zip_path)
# test_data = read_csv_from_zip(test_zip_path)

# Importar las librerias necesarias
import pandas as pd

# Leer los datasets
train_data = pd.read_csv("../files/input/train_data.csv.zip", index_col = False, compression = "zip")
test_data = pd.read_csv("../files/input/test_data.csv.zip", index_col = False, compression = "zip")

#### Paso 1

Realizar la limpieza de los datasets:

In [2]:
# Renombrar la columna "default payment next month" a "default".
train_data = train_data.rename(columns = {'default payment next month': 'default'})
test_data = test_data.rename(columns = {'default payment next month': 'default'})

# Remover la columna "ID".
train_data = train_data.drop(columns = ['ID'])
test_data = test_data.drop(columns = ['ID'])

# Eliminar los registros con informacion no disponible.
train_data = train_data.dropna()
test_data = test_data.dropna()

# Modificar la columna 'EDUCATION' para valores > 4, agrupar en "others"
train_data['EDUCATION'] = train_data['EDUCATION'].apply(lambda x: 4 if x > 4 else x)
test_data['EDUCATION'] = test_data['EDUCATION'].apply(lambda x: 4 if x > 4 else x)

#### Paso 2

Dividir los datasets en x_train, y_train, x_test, y_tes:

In [3]:
# Separar en (X) variables y variable objetivo (y)

# Conjunto de entrenamiento
X_train = train_data.drop(columns = ['default'])
y_train = train_data['default']

# Conjunto de prueba
X_test = test_data.drop(columns = ['default'])
y_test = test_data['default']

#### Paso 3

Crear un pipeline para el modelo de clasificación. Este pipeline debe contener las siguientes capas:

##### Transformar las variables categoricas usando el método one-hot-encoding:

In [4]:
# Importar las librerias necesarias
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# Seleccionar las columnas categóricas
categorical_columns = ['SEX', 'EDUCATION', 'MARRIAGE']

# Convertir las columnas que representan categorías a tipo 'category'
X_train[categorical_columns] = X_train[categorical_columns].astype('category')
X_test[categorical_columns] = X_test[categorical_columns].astype('category')

# Definir el transformador para las variables categóricas (One-Hot Encoding)
preprocessor = ColumnTransformer(
    transformers = [
        ('cat', OneHotEncoder(), categorical_columns)
    ],
    remainder = 'passthrough'  # Mantener las columnas numéricas sin cambios
)

##### Ajustar un modelo de bosques aleatorios (random forest):

In [5]:
# Importar las librerias necesarias
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

# Crear el pipeline con preprocesamiento y modelo de Random Forest
pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ('classifier', RandomForestClassifier(random_state = 38))
])

#### Paso 4

Optimizar los hiperparametros del pipeline usando validación cruzada. Usar 10 splits para la validación cruzada. Usar la función de precisión balanceada para medir la precisión del modelo:

In [12]:
# Importar las librerias necesarias
from sklearn.model_selection import GridSearchCV

# Definir los parámetros que se van a probar en el GridSearch
param_grid = {
    'classifier__n_estimators': [200],                  # Número de árboles
    'classifier__max_features': ['log2', 'sqrt', None], # Número de características a considerar en cada split
    'classifier__max_depth': [None],                    # Profundidad máxima de los árboles
    'classifier__min_samples_split': [10],              # Mínimo número de muestras para dividir un nodo
    'classifier__min_samples_leaf': [1, 2, 4],          # Mínimo número de muestras en un nodo hoja
    'classifier__bootstrap': [True]                     # Si se usa bootstrap en la creación de los árboles
}

# Crear el objeto GridSearchCV con validación cruzada de 10 splits
grid_search = GridSearchCV(
    pipeline,                      # El pipeline que definimos previamente
    param_grid,                    # Los hiperparámetros a probar
    cv = 10,                       # Número de splits para validación cruzada
    scoring = "balanced_accuracy", # Usamos precisión balanceada
    n_jobs = -1,                   # Usar todos los núcleos disponibles
    verbose = 1                    # Mostrar el progreso
)

# Ajustar el modelo con el conjunto de entrenamiento
grid_search.fit(X_train, y_train)

# Guardar el mejor modelo encontrado
best_model = grid_search.best_estimator_

# Ver los mejores parámetros encontrados por GridSearch
print(f"Mejores hiperparámetros: {grid_search.best_params_}")

Fitting 10 folds for each of 9 candidates, totalling 90 fits
Mejores hiperparámetros: {'classifier__bootstrap': True, 'classifier__max_depth': None, 'classifier__max_features': 'sqrt', 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 10, 'classifier__n_estimators': 200}


##### Paso 5

Guardar el modelo como "files/models/model.pkl":

In [13]:
# Importar la librería pickle
import pickle

model_path = "../files/models/model.pkl"

# Guardar el modelo entrenado como un archivo .pkl
with open(model_path, "wb") as model_file:
    pickle.dump(best_model, model_file)

print(f"Modelo guardado en '{model_path}'")

Modelo guardado en '../files/models/model.pkl'


##### Paso 6

Calcular las metricas de precisión, precisión balanceada, recall, y f1-score para los conjuntos de entrenamiento y prueba. Guardarlas en el archivo files/output/metrics.json. Cada fila del archivo es un diccionario con las metricas de un modelo. 

Este diccionario tiene un campo para indicar si es el conjunto de entrenamiento o prueba. Por ejemplo:

{'dataset': 'train', 'precision': 0.8, 'balanced_accuracy': 0.7, 'recall': 0.9, 'f1_score': 0.85}

{'dataset': 'test', 'precision': 0.7, 'balanced_accuracy': 0.6, 'recall': 0.8, 'f1_score': 0.75}

In [14]:
# Importar las librerias necesarias
from sklearn.metrics import precision_score, balanced_accuracy_score, recall_score, f1_score
import json

# Hacer predicciones en los conjuntos de entrenamiento y prueba
train_preds = best_model.predict(X_train)
test_preds = best_model.predict(X_test)

# Calcular las métricas para el conjunto de entrenamiento
train_metrics = {
    'type': 'metrics',
    'dataset': 'train',
    'precision': precision_score(y_train, train_preds),
    'balanced_accuracy': balanced_accuracy_score(y_train, train_preds),
    'recall': recall_score(y_train, train_preds),
    'f1_score': f1_score(y_train, train_preds)
}

# Calcular las métricas para el conjunto de prueba
test_metrics = {
    'type': 'metrics',
    'dataset': 'test',
    'precision': precision_score(y_test, test_preds),
    'balanced_accuracy': balanced_accuracy_score(y_test, test_preds),
    'recall': recall_score(y_test, test_preds),
    'f1_score': f1_score(y_test, test_preds)
}

# Crear una lista con las métricas
metrics = [train_metrics, test_metrics]

# Ruta del archivo de salida
output_file_path = "../files/output/metrics.json"

# Guardar las métricas en un archivo JSON
with open(output_file_path, "w") as json_file:
    json.dump(metrics, json_file)

print(f"Métricas guardadas en '{output_file_path}'")

Métricas guardadas en '../files/output/metrics.json'


##### Paso 7
 
Calcular las matrices de confusión para los conjuntos de entrenamiento y prueba. Guardarlas en el archivo files/output/metrics.json. 

Cada fila del archivo es un diccionario con las metricas de un modelo de entrenamiento o prueba. Por ejemplo:

{'type': 'cm_matrix', 'dataset': 'train', 'true_0': {"predicted_0": 15562, "predicte_1": 666}, 'true_1': {"predicted_0": 3333, "predicted_1": 1444}}
 
{'type': 'cm_matrix', 'dataset': 'test', 'true_0': {"predicted_0": 15562, "predicte_1": 650}, 'true_1': {"predicted_0": 2490, "predicted_1": 1420}}

In [15]:
# Importar las librerias necesarias
from sklearn.metrics import confusion_matrix
import json

# Calcular las predicciones para los conjuntos de entrenamiento y prueba
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

# Calcular las matrices de confusión
cm_train = confusion_matrix(y_train, y_train_pred)
cm_test = confusion_matrix(y_test, y_test_pred)

# Convertir las matrices a tipos nativos de Python (int)
cm_train = cm_train.astype(int).tolist()
cm_test = cm_test.astype(int).tolist()

# Formatear las matrices de confusión en el formato solicitado
cm_train_dict = {
    'type': 'cm_matrix',
    'dataset': 'train',
    'true_0': {"predicted_0": cm_train[0][0], "predicted_1": cm_train[0][1]},
    'true_1': {"predicted_0": cm_train[1][0], "predicted_1": cm_train[1][1]}
}

cm_test_dict = {
    'type': 'cm_matrix',
    'dataset': 'test',
    'true_0': {"predicted_0": cm_test[0][0], "predicted_1": cm_test[0][1]},
    'true_1': {"predicted_0": cm_test[1][0], "predicted_1": cm_test[1][1]}
}

# Ruta del archivo de salida
output_file_path = "../files/output/metrics.json"

# Leer las métricas existentes del archivo JSON
with open(output_file_path, 'r') as json_file:
    existing_metrics = json.load(json_file)

# Agregar las matrices de confusión a las métricas existentes
existing_metrics.append(cm_train_dict)
existing_metrics.append(cm_test_dict)

# Guardar las métricas actualizadas en el archivo JSON
with open(output_file_path, 'w') as json_file:
    json.dump(existing_metrics, json_file)

print(f"Matrices de confusión guardadas en '{output_file_path}'")

Matrices de confusión guardadas en '../files/output/metrics.json'


In [17]:
existing_metrics

[{'type': 'metrics',
  'dataset': 'train',
  'precision': 0.9717293233082707,
  'balanced_accuracy': 0.8388718817938129,
  'recall': 0.6835202030886397,
  'f1_score': 0.8025335320417287},
 {'type': 'metrics',
  'dataset': 'test',
  'precision': 0.6586620926243568,
  'balanced_accuracy': 0.6730886930577491,
  'recall': 0.4023048716605553,
  'f1_score': 0.4995121951219512},
 {'type': 'cm_matrix',
  'dataset': 'train',
  'true_0': {'predicted_0': 16179, 'predicted_1': 94},
  'true_1': {'predicted_0': 1496, 'predicted_1': 3231}},
 {'type': 'cm_matrix',
  'dataset': 'test',
  'true_0': {'predicted_0': 6693, 'predicted_1': 398},
  'true_1': {'predicted_0': 1141, 'predicted_1': 768}}]